In [ ]:
import urllib.request
from time import sleep
from bs4 import BeautifulSoup
import MeCab
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def url2text(urls):
    article = []
    for url in urls:
        print(url + "のデータを取得しています")
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html,"html.parser")
        soup_article = soup.find("div", class_="mw-parser-output")
        soup_article= soup_article.get_text()
        sleep(3)
    return article

In [ ]:
def mplg(article):
    word_list = ""
    m = MeCab.Tagger () # ←Taggerって何？
    m_article = m.parse (article) # parceって何?

    for row in m_article.split("\n"):
        word =row.split("\t")[0]#タブ区切りになっている１つ目を取り出す。ここには形態素が格納されている
        if word == "EOS": # EOSって何？
            break
        else:
            pos = row.split("\t")[1]#タブ区切りになっている2つ目を取り出す。ここには品詞が格納されている
            slice = pos[:2] # この書き方不明
            if slice == "名詞":
                word_list = word_list +" "+ word
    return word_list

In [ ]:
def tfidf(word_list):
    docs = np.array(word_list)#Numpyの配列に変換する
    #単語を配列ベクトル化して、TF-IDFを計算する
    vecs = TfidfVectorizer(
                token_pattern=u'(?u)\\b\\w+\\b'#文字列長が 1 の単語を処理対象に含めることを意味します。
                ).fit_transform(docs) # TfidfVectorizer,.fit_transformを調べる。
    vecs = vecs.toarray()
    return vecs

In [ ]:
def cossim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# コサイン類似度とは？

In [ ]:
def main():
    article = url2text([
            "http://mensnote.boy.jp/document/html.html"
            ])
    word_list = []
    for article in article:
        word_list.append(mplg(article))
    vecs = tfidf(word_list)

    print("HTMLとPython")
    print(cossim(vecs[1],vecs[0]))

main()